## In this notebook: breast cancer diagnosis using multiple machine learning techniques:
1. XGBoost (dominates hackathons and challenges such as Kaggle competitions)
2. Random forest classifier
2. Support vector machine
4. k-nearest neighbors
4. Deep learning
5. Principal component analysis (visualization)

In [ ]:
import sys

# Matrix operations
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Data handling
import pandas as pd

# Support vector machine model
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix
from xgboost import plot_tree, plot_importance, to_graphviz
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.externals.six import StringIO  
import pydot 


# Data preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# SMOTE sampling
import imblearn
from imblearn.over_sampling import SMOTE

# PCA for visualization purposes
import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# deep learning
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from IPython.display import Image
from keras.utils.np_utils import to_categorical


In [ ]:
print('sklearn version is {}.'.format(sklearn.__version__))

### Data visualization

In [ ]:
filepath = "breast-cancer-wisconsin-original.csv"
attributes = ['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean',
              'compactness_mean','concavity_mean','concave_points_mean','symmetry_mean','fractal_dimension_mean',
              'radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se',
              'concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst',
              'perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave_points_worst',
              'symmetry_worst','fractal_dimension_worst']

df = pd.read_csv(
    filepath_or_buffer = filepath,
    names = attributes)

df.head(5)

### There is relatively a small amount of data so we will see how well the models perform

In [ ]:
print(df.shape)

In [ ]:
df.drop(
    labels = ['id'],
    axis = 1,
    inplace = True)

df['diagnosis'] = pd.get_dummies(df['diagnosis'])

df.head(3)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 8]
df.describe()

In [ ]:
df.groupby('diagnosis').count()

In [ ]:
y = np.array(df['diagnosis'])
df = np.array(df.drop(['diagnosis'], 1))

### Oversample with SMOTE (if necessary)

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE

X = np.array(df)

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X,
    y,
    test_size = 0.25)

sm = SMOTE(
    sampling_strategy = 'not majority',
    k_neighbors = 5,
    n_jobs = 1,
    random_state = 12,
    ratio = 1.0)

X_res, y_res = sm.fit_sample(
    X_train,
    y_train)

In [ ]:
print('X_train length: ' + str(len(X_train)))
print('X_test length: ' + str(len(X_test)))
print('X_res length: ' + str(len(X_res)))

## XGBoost
XGBoost, just like a random forest classifier, is tree based and does not require feature scaling. Additionally, XGBoost uses boosting trees as opposed to gradient descent as another reason scaling is unncessary. 

- https://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf
- https://github.com/dmlc/xgboost/issues/357
- https://pypi.org/project/xgboost/ 

Use grid search to find the best parameters (won't experiment with many and put early stopping at 10 rounds for the sake of time)

In [ ]:
import xgboost
import sklearn
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

model = XGBClassifier()

parameters = {
    'n_estimators':[10, 20, 50, 100, 500, 1000],
    'max_depth': [3,4,5,10,20],
    'learning_rate': [1e-1, 1e-2, 1e-3, 1e-4]}

model = GridSearchCV(
    model,
    parameters,
    cv = 10,
    return_train_score = True,
    iid = False)

model.fit(
    X_train,
    y_train,
    eval_set = [(X_train, y_train), (X_test, y_test)],
    early_stopping_rounds = 10,
    eval_metric=["error", "logloss"],
    verbose = True)

Best parameters according to grid search

In [ ]:
print(model.best_estimator_)

In [ ]:
optimized_xgb = model.best_estimator_

optimized_xgb.fit(
    X_train,
    y_train, 
    eval_metric=["error", "logloss"])

prediction_xgb = optimized_xgb.predict(X_test)

print('Test accuracy: ' + str(accuracy_score(y_test,prediction_xgb)))
print('')
print('')

print(classification_report(
    y_test,
    prediction_xgb))

In [ ]:
results = optimized_xgb.evals_result()

epochs = len(results['validation_0']['error'])

x_axis = range(0, epochs)

fig, ax = plt.subplots()

ax.plot(
    x_axis,
    results['validation_0']['logloss'],
    label='Train')

ax.plot(
    x_axis,
    results['validation_1']['logloss'],
    label='Test')

ax.legend()
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()

fig, ax = plt.subplots()
ax.plot(
    x_axis,
    results['validation_0']['error'],
    label='Train')

ax.plot(
    x_axis,
    results['validation_1']['error'],
    label='Test')

ax.legend()
plt.ylabel('Error')
plt.title('XGBoost Error')
plt.show()

In [ ]:
import matplotlib as mpl
import pydot
mpl.rcParams['figure.dpi'] = 500

from sklearn.metrics import mean_absolute_error
from xgboost import plot_tree, plot_importance, to_graphviz

predictions = optimized_xgb.predict(X_test)

print('Test accuracy: ' + str(accuracy_score(y_test,predictions)))
print('')
print('')
print(classification_report(
    y_test,
    predictions))

plot_importance(optimized_xgb)

plot_tree(optimized_xgb, num_trees=2, rankdir='LR')
plt.show()

## Random Forest Classifier

https://stackoverflow.com/questions/8961586/do-i-need-to-normalize-or-scale-data-for-randomforest-r-package

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators = 500,
    random_state = 42) 

model.fit(
    X_train,
    y_train)

predictions = model.predict(X_test)

In [ ]:
print('Test accuracy: ' + str(accuracy_score(y_test,predictions)))
print('')
print('')
print(classification_report(
    y_test,
    predictions))

In [ ]:
total = len(predictions)
correct = 0
for pred, actual in zip(predictions,y_test):
    if pred == actual:
        correct += 1
        
print('Prediction accuracy: ' + str(correct/float(total)))
print('')

In [ ]:
from sklearn import tree
from sklearn.externals.six import StringIO  
import pydot 

dot_data = StringIO() 
tree.export_graphviz(model[1], out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 

graph[0].write_pdf("tree.pdf") 

## Support Vector Machine (using grid search)

Support vector machine, being distance based, requires the data to be scaled so no feature takes precedence over another. Scale the training data between 0,1 and scale the test data with the training data scaling parameters.

In [ ]:
print("X_train before scaling: ")
print(X_train)
print('')
print("X_test before scaling: ")
print(X_test)

In [ ]:
from sklearn import preprocessing 

scaler = preprocessing.MinMaxScaler() 

X_train = scaler.fit_transform(X_train)

print(scaler.data_max_)
print(scaler.data_min_)

X_test = scaler.transform(X_test)

In [ ]:
print("X_train after scaling: ")
print(X_train)
print('')
print("X_test after scaling: ")
print(X_test)

Use grid search to find the best kernel and value of C

In [ ]:
clf = SVC(gamma = 'auto')

parameters = {
    'kernel':('linear', 'rbf'),
    'C':[0.1, 0.5, 1, 2, 5, 10, 100, 1000]}

clf = GridSearchCV(
    clf, parameters,
    cv = 10,
    return_train_score = True,
    iid = False,
    verbose = True)

clf.fit(
    X_train,
    y_train)

In [ ]:
sorted(clf.cv_results_.keys())

In [ ]:
clf.best_estimator_

In [ ]:
optimized_svm = clf.best_estimator_

optimized_svm.fit(
    X_train,
    y_train)

cv_results = model_selection.cross_val_score(
    optimized_svm,
    X_train,
    y_train,
    cv = 10,
    scoring='accuracy')

prediction_svm = optimized_svm.predict(X_test)

print('Test accuracy: ' + str(accuracy_score(y_test,prediction_svm)))
print('')
print('')
print(classification_report(
    y_test,
    prediction_svm))

## To visualize the hyperplanes, use PCA to reduce to 2 dimensions

In [ ]:
pca = PCA(n_components = 2)

In [ ]:
X_train = StandardScaler().fit_transform(X_train)

principalComponents = pca.fit_transform(X_train)

In [ ]:
optimized_svm.fit(
    principalComponents,
    y_train)

### Plot the decision boundary from grid search

In [ ]:
plt.scatter(principalComponents[:, 0],
            principalComponents[:, 1],
            s = 30,
            cmap = plt.cm.Paired)

ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

xx = np.linspace(
    start = xlim[0],
    stop = xlim[1],
    num = 30)

yy = np.linspace(
    start = ylim[0],
    stop = ylim[1],
    num = 30)

YY, XX = np.meshgrid(
    yy,
    xx)

xy = np.vstack([XX.ravel(), YY.ravel()]).T

Z = clf.decision_function(xy).reshape(XX.shape)

ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['-', '-', '-'])

ax.scatter(optimized_svm.support_vectors_[:, 0], optimized_svm.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')
plt.title('Linear Decision Boundary')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()


## Out of curiousity, the radial basis function boundary is plotted below

In [ ]:
import matplotlib.pyplot as plt

clf = SVC(
    gamma = 'auto',
    C = 1.0,
    kernel = 'rbf')

clf.fit(principalComponents,
        y_train)

h = 0.2
x_min, x_max = principalComponents[:,0].min() - 1, principalComponents[:, 0].max() + 1
y_min, y_max = principalComponents[:,1].min() - 1, principalComponents[:, 1].max() + 1

xx, yy = np.meshgrid(
    np.arange(x_min, x_max, h),
    np.arange(y_min, y_max, h))

Z = clf.predict(
    np.c_[xx.ravel(),
          yy.ravel()])

Z = Z.reshape(xx.shape)

plt.contourf(
    xx,
    yy,
    Z,
    cmap = plt.cm.coolwarm,
    alpha = 0.8)

plt.scatter(
    principalComponents[:,0],
    principalComponents[:,1],
    c = y_train)

plt.title('Radial Basis Function Kernel Boundaries')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()


## K-Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors = 15)

neigh.fit(
    X_train,
    y_train) 

results = neigh.predict(X_test)

print('Test accuracy: ' + str(accuracy_score(y_test,results)))
print('')
print('')
print(classification_report(
    y_test,
    results))

### Plot adapted from sklearn: https://scikit-learn.org/stable/auto_examples/neighbors/plot_nca_classification.html#sphx-glr-auto-examples-neighbors-plot-nca-classification-py

In [ ]:
from sklearn import neighbors
from matplotlib.colors import ListedColormap

h = .02

cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

for weights in ['uniform', 'distance']:

    clf = neighbors.KNeighborsClassifier(
        n_neighbors = 15,
        weights = weights)
    
    clf.fit(
        principalComponents,
        y_train)

    x_min, x_max = principalComponents[:, 0].min() - 1, principalComponents[:, 0].max() + 1
    y_min, y_max = principalComponents[:, 1].min() - 1, principalComponents[:, 1].max() + 1
    
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, h),
        np.arange(y_min, y_max, h))
  
    Z = clf.predict(
        np.c_[xx.ravel(),
        yy.ravel()])

    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(
        xx,
        yy,
        Z,
        cmap = cmap_light)

    plt.scatter(
        principalComponents[:, 0],
        principalComponents[:, 1],
        c = y_train,
        cmap=cmap_bold,
        edgecolor='k', s = 20)
    
    plt.xlim(
        xx.min(),
        xx.max())
    
    plt.ylim(
        yy.min(),
        yy.max())
    
    plt.title("2 Class Classification (k = 15)")
              

plt.show()

## Deep learning

In [ ]:
seed = 15

# split for train, test
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X,
    y,
    test_size=0.3,
    random_state = seed,
    shuffle = True)

Y_train = to_categorical(y_train,
                         num_classes = None)

Y_test = to_categorical(y_test,
                        num_classes = None)

# split for train, validation
X_train, X_val, Y_train, Y_val = model_selection.train_test_split(
    X_train,
    Y_train,
    test_size=0.3,
    random_state = seed,
    shuffle = True)

# Upsampling
sm = SMOTE(
    sampling_strategy='not majority',
    k_neighbors = 10,
    n_jobs = 1,
    random_state = 12,
    ratio = 0.7)

X_res, Y_res = sm.fit_sample(X_train, Y_train)

Y_res = to_categorical(Y_res,
                        num_classes = None)


In [ ]:
input_layer = Input(shape=(30,))

x = Dense(
    units = 96,
    kernel_initializer='glorot_uniform',
    use_bias = True,
    bias_initializer='zeros')(input_layer)
x = BatchNormalization()(x)
x = Activation('softmax')(x)

skip1 = x

x = Dense(
    units = 96,
    kernel_initializer='glorot_uniform',
    use_bias = True,
    bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x = keras.layers.Add()([x, skip1])
x = Activation('relu')(x)

skip2 = x 

x = Dropout(rate = 0.3)(x)

x = Dense(
    units = 96,
    kernel_initializer='glorot_uniform',
    use_bias = True,
    bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x = keras.layers.Add()([x, skip2])
x = Activation('relu')(x)

x = Dropout(rate = 0.3)(x)

x = Dense(
    units = 96,
    kernel_initializer='glorot_uniform',
    use_bias = True,
    bias_initializer='zeros')(input_layer)
x = BatchNormalization()(x)
x = keras.layers.Add()([x, skip1])
x = keras.layers.Add()([x, skip2])
x = Activation('relu')(x)

x = Dropout(rate = 0.3)(x)

y = Dense(
    units = 2,
    activation='softmax')(x)

model = Model(
    inputs = input_layer,
    outputs = y)

model.summary()

### Model performed on oversampled data

In [ ]:
opt = SGD(lr= 2e-4)

model.compile(
    loss = "categorical_crossentropy",
    optimizer = opt,
    metrics = ['accuracy'])

reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.5,
    patience = 5, 
    min_lr = 5e-7,
    verbose = 1)

early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    verbose = 1,
    restore_best_weights = True)

history = model.fit(
    X_res,
    Y_res,                   
    epochs = 300, 
    batch_size = 4,
    verbose = 1,
    validation_data = (X_val, Y_val),
    callbacks = [reduce_lr,
                early_stopping])

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
predictions = model.predict(X_test)
print('Test accuracy: ' + str(accuracy_score(np.argmax(Y_test, axis=1),np.argmax(predictions, axis=1))))
print('')
print('')
print(classification_report(np.argmax(Y_test, axis=1),np.argmax(predictions, axis=1)))

In [ ]:
del model

## Model performed on non-oversampled data

In [ ]:
input_layer = Input(shape=(30,))

x = Dense(
    units = 96,
    kernel_initializer='glorot_uniform',
    use_bias = True,
    bias_initializer='zeros')(input_layer)
x = BatchNormalization()(x)
x = Activation('softmax')(x)

skip1 = x

x = Dense(
    units = 96,
    kernel_initializer='glorot_uniform',
    use_bias = True,
    bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x = keras.layers.Add()([x, skip1])
x = Activation('relu')(x)

skip2 = x 

x = Dropout(rate = 0.3)(x)

x = Dense(
    units = 96,
    kernel_initializer='glorot_uniform',
    use_bias = True,
    bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x = keras.layers.Add()([x, skip2])
x = Activation('relu')(x)

x = Dropout(rate = 0.3)(x)

x = Dense(
    units = 96,
    kernel_initializer='glorot_uniform',
    use_bias = True,
    bias_initializer='zeros')(input_layer)
x = BatchNormalization()(x)
x = keras.layers.Add()([x, skip1])
x = keras.layers.Add()([x, skip2])
x = Activation('relu')(x)

x = Dropout(rate = 0.3)(x)

y = Dense(
    units = 2,
    activation='softmax')(x)

model = Model(
    inputs = input_layer,
    outputs = y)

model.summary()

opt = SGD(lr= 2e-4)

model.compile(
    loss = "categorical_crossentropy",
    optimizer = opt,
    metrics = ['accuracy'])

reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.5,
    patience = 5, 
    min_lr = 5e-7,
    verbose = 1)

early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    verbose = 1,
    restore_best_weights = True)

history = model.fit(
    X_res,
    Y_res,                   
    epochs = 300, 
    batch_size = 4,
    verbose = 1,
    validation_data = (X_val, Y_val),
    callbacks = [reduce_lr,
                early_stopping])

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
predictions = model.predict(X_test)
print('Test accuracy: ' + str(accuracy_score(np.argmax(Y_test, axis=1),np.argmax(predictions, axis=1))))
print('')
print('')
print(classification_report(np.argmax(Y_test, axis=1),np.argmax(predictions, axis=1)))

In [ ]:
del model